Code Author: Ha Eungyeom (eungyeom_ha@yonsei.ac.kr)        
This code is developed for training and evaluating a Faster-RCNN model on the HOD dataset

In [1]:
!pwd

/home/oks/people/egha


### Applying Inference and Evaluation on the Test Dataset : Normal Cases
* Creating the test Dataset and DataLoader and calling single_gpu_test() to return the inference results. An error occurs in single_gpu_test() if batch_size is not set to 1.

In [2]:
from mmcv.runner import HOOKS, Hook

# Defining a new hook class for saving the best checkpoint based on the evaluation metric.
@HOOKS.register_module()
class SaveBestCheckpointHook(Hook):
    def __init__(self, out_dir, metric='bbox_mAP_50', save_optimizer=True): #  metric='bbox_mAP
        self.out_dir = out_dir
        self.metric = metric
        self.save_optimizer = save_optimizer
        self.best_score = 0.0

    def after_train_epoch(self, runner):
        # Check and save the best model checkpoint after each training epoch based on the evaluation metric.
        if not self.every_n_epochs(runner, 1):
            return
        from mmcv.runner import save_checkpoint
        if runner.log_buffer.output.get(self.metric, 0) > self.best_score:
            self.best_score = runner.log_buffer.output[self.metric]
            save_checkpoint(runner.model, self.out_dir, optimizer=self.save_optimizer)

/home/oks/anaconda3/envs/egha-mm/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [ ]:
name = '/rcnn_normal/'

In [ ]:
from mmcv import Config
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.coco import CocoDataset
from mmdet.apis import set_random_seed

# Registering a new dataset class based on the CocoDataset class.
@DATASETS.register_module(force=True)
class HOD(CocoDataset):
    CLASSES = ('alcohol', 'insulting_gesture', 'blood', 'cigarette', 'gun', 'knife') 

config_file = './mmdetection/configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py'

cfg = Config.fromfile(config_file)

# Modifying the dataset-related configurations. 
cfg.dataset_type = 'HOD'
cfg.data_root = './faster_rcnn_data' + name

# Updating the type, data_root, ann_file, img_prefix configurations for the train, val, and test datasets.. 
cfg.data.train.type = 'HOD'
cfg.data.train.data_root = './faster_rcnn_data'+ name
cfg.data.train.ann_file = 'train.json'
cfg.data.train.img_prefix = 'JPEGImages'

cfg.data.val.type = 'HOD'
cfg.data.val.data_root = './faster_rcnn_data' + name
cfg.data.val.ann_file = 'val.json'
cfg.data.val.img_prefix = 'JPEGImages'

cfg.data.test.type = 'HOD'
cfg.data.test.data_root = './faster_rcnn_data' + '/rcnn_hard/' # hard cases 
cfg.data.test.ann_file = 'test.json'
cfg.data.test.img_prefix = 'JPEGImages'

# Updating the number of classes.
cfg.model.roi_head.bbox_head.num_classes = 6
# Pretrained model
cfg.load_from = './mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'

# Setting the directory for saving the logs of training weights.
cfg.work_dir = './tutorial_exps_normal'

# Updating the learning rate configurations.
cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None
cfg.log_config.interval = 2000

# For CocoDataset, the metric should be set to 'bbox' (instead of mAP). Setting it to 'bbox' calculates mAP over a range of IoU thresholds (0.5 to 0.95)
cfg.evaluation.metric = 'bbox'
cfg.evaluation.interval = 2000
cfg.checkpoint_config.interval = 5
cfg.custom_hooks = [dict(type='SaveBestCheckpointHook', out_dir=cfg.work_dir, metric='bbox_mAP', save_optimizer=True)]

# Due to a bug(?), set the samples_per_gpu to 1 for test dataset evaluation. In the data loader, the batch size is determined by the number of GPUs.
cfg.data.samples_per_gpu = 1

# Due to an error where loading the config twice causes the lr_config's policy to disappear, it's set again here.
cfg.lr_config.policy='step'
# Set the seed for more reproducible results.
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.custom_hooks = [dict(type='SaveBestCheckpointHook', out_dir=cfg.work_dir, metric='bbox_mAP', save_optimizer=True)]

# Addition by Eunkyeom
# Change the evaluation metric since we use a customized dataset.
cfg.device = 'cuda'
cfg.runner.max_epochs = 150

print(cfg.pretty_text)

In [ ]:
cfg.dump('./tutorial_exps_normal/HOD_faster_rcnn_conf_normal.py')

In [ ]:
!mkdir -p ./show_test_output_normal # Create a directory to save the results

### Setting up the test dataset and dataloader separately, and loading the trained checkpoint model for testing.

In [8]:
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
# Creating the test Dataset and DataLoader.
# Unlike when creating the train dataset, do not wrap build_dataset() in a list. 
dataset = build_dataset(cfg.data.test) 
data_loader = build_dataloader(
        dataset,
        # Must set the samples_per_gpu argument value to 1
        # samples_per_gpu=cfg.data.samples_per_gpu,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

# Must ensure that the 'img' key value is output as a tensor.
next(iter(data_loader))

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


{'img_metas': [DataContainer([[{'filename': '/home/oks/people/egha/ob_code_faster_rcnn/faster_rcnn_data/rcnn_hard/JPEGImages/21_0158.jpg', 'ori_filename': '21_0158.jpg', 'ori_shape': (232, 391, 3), 'img_shape': (791, 1333, 3), 'pad_shape': (800, 1344, 3), 'scale_factor': array([3.409207 , 3.4094827, 3.409207 , 3.4094827], dtype=float32), 'flip': False, 'flip_direction': None, 'img_norm_cfg': {'mean': array([123.675, 116.28 , 103.53 ], dtype=float32), 'std': array([58.395, 57.12 , 57.375], dtype=float32), 'to_rgb': True}}]])],
 'img': [tensor([[[[1.8722, 1.8722, 1.8722,  ..., 0.0000, 0.0000, 0.0000],
            [1.8722, 1.8722, 1.8722,  ..., 0.0000, 0.0000, 0.0000],
            [1.8722, 1.8722, 1.8550,  ..., 0.0000, 0.0000, 0.0000],
            ...,
            [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
            [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
            [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
  
           [[1.8683, 1.8683, 1.8

In [10]:
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

checkpoint_file = './tutorial_exps_normal/latest.pth'

# Using the checkpoint saved model file to create a model, using the above updated config.
# Load model
model_ckpt = init_detector(cfg, checkpoint_file, device='cuda:0')

load checkpoint from local path: /home/oks/people/egha/ob_code_faster_rcnn/normal_fast//tutorial_exps_normal/latest.pth


In [ ]:
from mmdet.apis import multi_gpu_test, single_gpu_test
from mmcv.parallel import MMDataParallel, MMDistributedDataParallel
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

model_ckpt = MMDataParallel(model_ckpt, device_ids=[0])

# Calling single_gpu_test() to perform inference on the test dataset. Must set the batch size to 1.
# The inference results are saved as visualized images in the /kaggle/working/show_test_output directory.
# Set the threshold properly to 0.5 !!!
outputs = single_gpu_test(model_ckpt, data_loader, True, './show_test_output_hard', 0.5)

### Checking the returned inference applied results of the test dataset and performing performance evaluation.

In [12]:
print('Result outputs type:', type(outputs))
print('Number of evaluated files:', len(outputs))
print('Type of the first evaluation result:', type(outputs[0]))
print('Number of CLASSES in the first evaluation result:', len(outputs[0]))
print('Type and shape of CLASS ID 0 in the first evaluation result:', type(outputs[0][0]), outputs[0][0].shape)

결과 outputs type: <class 'list'>
evalution 된 파일의 갯수: 255
첫번째 evalutation 결과의 type: <class 'list'>
첫번째 evaluation 결과의 CLASS 갯수: 6
첫번째 evaluation 결과의 CLASS ID 0의 type과 shape <class 'numpy.ndarray'> (0, 5)


In [13]:
metric = dataset.evaluate(outputs, metric='bbox', classwise=True) # AP0.05 ~ 0.95
print(metric)


Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.17s).
Accumulating evaluation results...
DONE (t=0.07s).

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.203
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.340
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.219
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.008
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.179
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.304
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.263
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.263
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.263
 Average Recall     (AR) @[ IoU=0.50:0.95

In [14]:
metric_5095 = dataset.evaluate(outputs, metric='bbox', classwise=True, iou_thrs = [0.5])
print(metric_5095)


Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.11s).
Accumulating evaluation results...
DONE (t=0.03s).

 Average Precision  (AP) @[ IoU=0.50:0.50 | area=   all | maxDets=100 ] = 0.340
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = -1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.50 | area= small | maxDets=1000 ] = 0.028
 Average Precision  (AP) @[ IoU=0.50:0.50 | area=medium | maxDets=1000 ] = 0.294
 Average Precision  (AP) @[ IoU=0.50:0.50 | area= large | maxDets=1000 ] = 0.501
 Average Recall     (AR) @[ IoU=0.50:0.50 | area=   all | maxDets=100 ] = 0.406
 Average Recall     (AR) @[ IoU=0.50:0.50 | area=   all | maxDets=300 ] = 0.406
 Average Recall     (AR) @[ IoU=0.50:0.50 | area=   all | maxDets=1000 ] = 0.406
 Average Recall     (AR) @[ IoU=0.50:0.

In [15]:
metric = dataset.evaluate(outputs, metric='bbox', classwise=True) # AP0.05 ~ 0.95
print(metric)


Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.18s).
Accumulating evaluation results...
DONE (t=0.07s).

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.203
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.340
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.219
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.008
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.179
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.304
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.263
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.263
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.263
 Average Recall     (AR) @[ IoU=0.50:0.95

In [16]:
metric = dataset.evaluate(outputs, metric='bbox', classwise=True, iou_thrs = [0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95]) # AP0.05 ~ 0.95
print(metric)


Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.13s).
Accumulating evaluation results...
DONE (t=0.06s).

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.624
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = -1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.449
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.661
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.695
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.695
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.695
 Average Recall     (AR) @[ IoU=0.50:0.

In [16]:
import json
import copy
import numpy as np

# Function to filter out the detection outputs based on confidence thresholds.
def filter_outputs_by_confidence(outputs, confidence_threshold):
    filtered_outputs = []
    for output_per_image in outputs:
        filtered_output_per_image = []
        for class_index, boxes in enumerate(output_per_image):
            if len(boxes) == 0:
                filtered_output_per_image.append(np.array([]))
                continue
            # Filtering the boxes based on the confidence score.
            filtered_boxes = boxes[boxes[:, 4] >= confidence_threshold]
            filtered_output_per_image.append(filtered_boxes)
        filtered_outputs.append(filtered_output_per_image)
    return filtered_outputs

# Calling single_gpu_test only o

# single_gpu_test를 한 번만 호출하여 결과를 얻습니다.
outputs = single_gpu_test(model_ckpt, data_loader, True, './show_test_output_hard', 0.5)

best_threshold = 0.0  # 가장 높은 mAP50 값을 가진 threshold를 저장할 변수
best_mAP50 = 0.0  # 가장 높은 mAP50 값을 저장할 변수

for con_fi in [0.1, 0.2, 0.3, 0.4, 0.5]:
    print(con_fi)
    # 이전에 얻은 outputs를 복사하여 필터링합니다.
    filtered_outputs = filter_outputs_by_confidence(copy.deepcopy(outputs), con_fi)
    metric = dataset.evaluate(filtered_outputs, metric='bbox', classwise=True)  # AP0.05 ~ 0.95
    metric_50 = dataset.evaluate(filtered_outputs, metric='bbox', classwise=True, iou_thrs=[0.5])
    
    # mAP50 값이 현재의 최고 값보다 높다면, 이 값을 최고 값으로 업데이트하고, threshold도 업데이트합니다.
    current_mAP50 = metric_50['bbox_mAP']
    if current_mAP50 > best_mAP50:
        best_mAP50 = current_mAP50
        best_threshold = con_fi
    
    print(metric)
    print()
    print(metric_50)
    print("=============================================================================================================")

# 가장 높은 mAP50 값을 가진 threshold를 문자열로 변환합니다.
best_threshold_str = str(best_threshold).replace('.', '_')
# 결과를 JSON 파일로 저장합니다.
with open(f'results_conf_{best_threshold_str}.json', 'w') as f:
    json.dump({'best_threshold': best_threshold, 'best_mAP50': best_mAP50}, f)

print(f"Best Threshold: {best_threshold}, Best mAP50: {best_mAP50}")


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 255/255, 5.6 task/s, elapsed: 46s, ETA:     0s0.1

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.15s).
Accumulating evaluation results...
DONE (t=0.06s).

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.198
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.328
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.215
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.008
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.173
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.298
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.252
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.252
 Average Recall     (AR) @[ IoU=0.50:0.